# Grasping analysis for robotic hands

This project was to create two classes, using the Sympy library, in order to calculate (symbolically) the Grasp matrix, **G**, and the Hand Jacobian, **J**, which are important for analyzing the Grasping of a robot's hand.

In [1]:
import sympy as sp
import numpy as np

In [5]:
class Grasp_hand:
    def __init__(self, Ri,n_c,n_q,cp, ciei,zi):
        """Classe for Grasping analysis in 3D

        Args:
            Ri (sympy.Matrix): rotactional matrix
            n_c (int): number of contact points of the hand
            n_q (int): number of joints of the hand
            cp (list): lsit of vectors (sympy.Matrix) for the J matrix
            ciei (list): list of vectors (sympy.Matrix) for the G matrix
            zi (list): list of vectors (sympy.Matrix) with the z axis for the J matrix
        """
        self.rx,self.ry,self.rz = sp.symbols("r_x r_y r_z")
        self.Ri = Ri
        self.S = sp.Matrix([[0, -self.rz, self.ry],[self.rz,0,-self.rx],[-self.ry,self.rx,0]])
        self.cp = cp
        self.n_c = n_c
        self.n_q = n_q
        self.ciei = ciei
        self.zi = zi
        self.R_barra = []
        self.G_T_barra = None
        self.J_barra = None
        self.J = None
        self.G_T = None

    def get_g_t_barra(self):
        """Generate the transpose of the G matrix before the selection matrix (H)

        Returns:
            sympy.Matrix: the transpose of the G matrix.
        """
        ind = sp.eye(3)
        zeros = sp.zeros(3)
        Gi_T_barra = []
        for i in range(self.n_c):
            Spi = self.S.subs([(self.rx,self.cp[i].row(0)[0]),(self.ry,self.cp[i].row(1)[0]),(self.rz,self.cp[i].row(2)[0])])
            Pi = ind.row_insert(0,zeros)
            Ri_barra = self.Ri[i].row_insert(0,zeros)
            Pi = Pi.col_insert(0,Spi.row_insert(0,ind))
            Ri_barra = Ri_barra.col_insert(0,zeros.row_insert(0,self.Ri[i]))
            self.R_barra.append(Ri_barra)
            Gi_T_barra.append((Ri_barra.T)*(Pi.T))
        self.G_T_barra = Gi_T_barra[-1].row_insert(0,Gi_T_barra[-2])
        for i in range(3,len(Gi_T_barra)+1):
            self.G_T_barra = self.G_T_barra.row_insert(0,Gi_T_barra[-i])
        return self.G_T_barra

    def d_i_j(self,joint,c=0,z=0):
        if joint:
            Sc = self.S.subs([(self.rx,c.row(0)[0]),(self.ry,c.row(1)[0]),(self.rz,c.row(2)[0])])
            return Sc*z
        else:
            return sp.Matrix([0,0,0])

    def k_i_j(self,joint,z=0):
        if joint:
            return z
        else:
            return sp.Matrix([0,0,0])

    def get_j_barra(self):
        """Generate the J matrix before the selection matrix (H)

        Returns:
            sympy.Matrix: the J matrix before the selection matrix (H).
        """
        Ji_barra = []
        for i in range(self.n_c):
            d = []
            k = []
            index = 0
            for j in range(len(self.ciei[i])):
                if self.ciei[i][j] != 0:
                    try:
                        d.append(self.d_i_j(True,c=self.ciei[i][j],z=self.zi[i][j]))
                        k.append(self.k_i_j(True,z=zi[i][j]))
                    except:
                        d.append(self.d_i_j(True,c=self.ciei[i][j]))
                        k.append(self.k_i_j(True))
                else:
                    d.append(self.d_i_j(False))
                    k.append(self.k_i_j(False))
                index += 1
            Zi = sp.Matrix([d,k])
            Ji_barra.append((self.R_barra[i].T)*(Zi))
        if self.n_c > 1:
            self.J_barra = Ji_barra[-1].row_insert(0,Ji_barra[-2])
            for i in range(3,len(Ji_barra)+1):
                self.J_barra = self.J_barra.row_insert(0,Ji_barra[-i])
        else:
            self.J_barra = Ji_barra[0]
        return self.J_barra

    def get_G_T(self,H):
        """Returns the transpose of the G matrix after the selection matrix (H)

        Args:
            H (sympy.Matrix): selection matrix

        Returns:
            sympy.Matrix: the transpose of the G matrix after the selection matrix (H)
        """
        self.G_T = H*self.G_T_barra
        return self.G_T

    def get_J(self,H):
        """Returns the J matrix after the selection matrix (H)

        Args:
            H (sympy.Matrix): selection matrix

        Returns:
            sympy.Matrix: the J matrix after the selection matrix (H)
        """
        self.J = H*self.J_barra
        return self.J

    def get_n_space(self,M):
        """Get the null space of a matrix M

        Args:
            M (sympy.Matrix): a matrix of the sympy

        Returns:
            sympy.Matrix: a matrix with the basis of the null space 
        """
        n = M.nullspace()
        if len(n) > 0:
            n_m = M.nullspace()[0]
            if len(n) > 1: 
                for i in range(1,len(M.nullspace())):
                    n_m = n_m.col_insert(i,M.nullspace()[i])
            return n_m
        else:
            return n
    
    def get_c_space(self,M):
        """Get the column space of a matrix M

        Args:
            M (sympy.Matrix): a matrix of the sympy

        Returns:
            sympy.Matrix: a matrix with the basis of the column space 
        """
        r = M.columnspace()
        if len(r) > 0:
            r_m = M.columnspace()[0]
            if len(r) > 1: 
                for i in range(1,len(M.columnspace())):
                    r_m = r_m.col_insert(i,M.columnspace()[i])
            return r_m
        else:
            return r
    


In [6]:
class Graps_hand_2d(Grasp_hand):
    def __init__(self, Ri, n_c, n_q, cp, ciei):
        """Classe for Grasping analysis in 2D

        Args:
            Ri (sympy.Matrix): rotactional matrix
            n_c (int): number of contact points of the hand
            n_q (int): number of joints of the hand
            cp (list): lsit of vectors (sympy.Matrix) for the J matrix
            ciei (list): list of vectors (sympy.Matrix) for the G matrix
            zi (list): list of vectors (sympy.Matrix) with the z axis for the J matrix
        """
        super().__init__(Ri, n_c, n_q, cp, ciei, zi=0)
        self.S = sp.Matrix([[-self.ry, self.rx]])

    def get_g_t_barra(self):
        ind = sp.eye(2)
        Gi_T_barra = []
        zeros = sp.zeros(1,2)
        for i in range(self.n_c):
            Spi = self.S.subs([(self.rx,self.cp[i].row(0)[0]),(self.ry,self.cp[i].row(1)[0])])
            Pi = zeros.T.row_insert(2,sp.Matrix([1]))
            Ri_barra = zeros.T.row_insert(2,sp.Matrix([1]))
            Pi = Pi.col_insert(0,ind.row_insert(2,Spi))
            Ri_barra = Ri_barra.col_insert(0,zeros.row_insert(0,self.Ri[i]))
            self.R_barra.append(Ri_barra)
            Gi_T_barra.append((Ri_barra.T)*(Pi.T))
        self.G_T_barra = Gi_T_barra[-1].row_insert(0,Gi_T_barra[-2])
        for i in range(3,len(Gi_T_barra)+1):
            self.G_T_barra = self.G_T_barra.row_insert(0,Gi_T_barra[-i])
        return self.G_T_barra


    def d_i_j(self,joint,c=0):
        if joint:
            Sc = self.S.subs([(self.rx,c.row(0)[0]),(self.ry,c.row(1)[0])])
            return Sc.T
        else:
            return sp.Matrix([0,0])

    def k_i_j(self,joint,z=0):
        if joint:
            return 1
        else:
            return 0

### Some examples of aplication

#### In 3D

In [14]:
theta, theta_1,theta_2, l1,l2,l3,l4,l5,l6,l7, r = sp.symbols("theta theta_1 theta_2 l_1 l_2 l_3 l_4 l_5 l_6 l_7 r", real=True)
rx,ry,rz = sp.symbols("r_x r_y r_z")
Ri = sp.Matrix([[-sp.cos(theta),sp.sin(theta),0],[-sp.sin(theta),-sp.cos(theta),0],[0,0,1]])

In [15]:
c1p = sp.Matrix([r*sp.cos(theta_1),r*sp.sin(theta_1),0])
c2p = sp.Matrix([r*sp.cos(theta_2),r*sp.sin(theta_2),0])
cp = [c1p,c2p]
theta_c = [theta_1, theta_2]

R_i = [Ri.subs(theta,t) for t in theta_c]

c1e1 = sp.Matrix([l2,l1,0])
c1e2 = sp.Matrix([l7,l3,0])
c2e3 = c2e4 = sp.Matrix([l4,l5,0])
c2e5 = sp.Matrix([l6,0,0])
z1 = sp.Matrix([0,0,1])
z2 = sp.Matrix([0,0,1])
z3 = sp.Matrix([0,1,0])
z4 = sp.Matrix([-np.sqrt(2)/2,np.sqrt(2)/2,0])
z5 = sp.Matrix([0,0,1])
ciei = [[c1e1,c1e2,0,0,0], [0,0,c2e3,c2e4,c2e5]]
zi = [[z1,z2,0,0,0],[0,0,z3,z4,z5]]

In [16]:
H = sp.Matrix([[sp.eye(4),sp.zeros(4,8)],[sp.zeros(4,6),sp.eye(4),sp.zeros(4,2)]])
H

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [7]:
example_3 = Grasp_hand(R_i,2,5,cp,ciei,zi)
g_t_barra = example_3.get_g_t_barra()
j_barra = example_3.get_j_barra()
G_T = example_3.get_G_T(H)
J = example_3.get_J(H)

In [8]:
G_T

Matrix([
[-cos(theta_1), -sin(theta_1), 0,              0,               0,                                      0],
[ sin(theta_1), -cos(theta_1), 0,              0,               0, -r*sin(theta_1)**2 - r*cos(theta_1)**2],
[            0,             0, 1, r*sin(theta_1), -r*cos(theta_1),                                      0],
[            0,             0, 0,  -cos(theta_1),   -sin(theta_1),                                      0],
[-cos(theta_2), -sin(theta_2), 0,              0,               0,                                      0],
[ sin(theta_2), -cos(theta_2), 0,              0,               0, -r*sin(theta_2)**2 - r*cos(theta_2)**2],
[            0,             0, 1, r*sin(theta_2), -r*cos(theta_2),                                      0],
[            0,             0, 0,  -cos(theta_2),   -sin(theta_2),                                      0]])

In [9]:
J

Matrix([
[-l_1*cos(theta_1) + l_2*sin(theta_1), -l_3*cos(theta_1) + l_7*sin(theta_1),             0,                                                                0,                0],
[ l_1*sin(theta_1) + l_2*cos(theta_1),  l_3*sin(theta_1) + l_7*cos(theta_1),             0,                                                                0,                0],
[                                   0,                                    0,             0,                                                                0,                0],
[                                   0,                                    0,             0,                                                                0,                0],
[                                   0,                                    0,             0,                                                                0, l_6*sin(theta_2)],
[                                   0,                                    0,             0,               

#### In 2D

In [10]:
c1p = sp.Matrix([r*sp.cos(theta_1),r*sp.sin(theta_1)])
c2p = sp.Matrix([r*sp.cos(theta_2),r*sp.sin(theta_2)])
cp = [c1p,c2p]
theta_c = [theta_1, theta_2]
Ri = sp.Matrix([[-sp.cos(theta),sp.sin(theta)],[-sp.sin(theta),-sp.cos(theta)]])
R_i = [Ri.subs(theta,t) for t in theta_c]

c1e1 = sp.Matrix([l2,l1])
c1e2 = sp.Matrix([l7,l3])
c2e3 = c2e4 = sp.Matrix([l4,l5])
c2e5 = sp.Matrix([l6,0])
ciei = [[c1e1,c1e2,0,0,0], [0,0,0,0,c2e5]]
H = sp.Matrix([[sp.eye(2), sp.zeros(2,1),sp.zeros(2,3)], [sp.zeros(2,3),sp.eye(2),sp.zeros(2,1)]])
H

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0]])

In [11]:
example_3_2d = Graps_hand_2d(R_i,2,5,cp,ciei)
g_t_barra = example_3_2d.get_g_t_barra()
j_barra = example_3_2d.get_j_barra()
G_T = example_3_2d.get_G_T(H)
J = example_3_2d.get_J(H)

In [12]:
G_T

Matrix([
[-cos(theta_1), -sin(theta_1),                                      0],
[ sin(theta_1), -cos(theta_1), -r*sin(theta_1)**2 - r*cos(theta_1)**2],
[-cos(theta_2), -sin(theta_2),                                      0],
[ sin(theta_2), -cos(theta_2), -r*sin(theta_2)**2 - r*cos(theta_2)**2]])

In [13]:
J

Matrix([
[ l_1*cos(theta_1) - l_2*sin(theta_1),  l_3*cos(theta_1) - l_7*sin(theta_1), 0, 0,                 0],
[-l_1*sin(theta_1) - l_2*cos(theta_1), -l_3*sin(theta_1) - l_7*cos(theta_1), 0, 0,                 0],
[                                   0,                                    0, 0, 0, -l_6*sin(theta_2)],
[                                   0,                                    0, 0, 0, -l_6*cos(theta_2)]])